In [ ]:
import geopandas as gpd
import pandas as pd
import folium
from branca.colormap import linear

In [ ]:
# Ruta del GeoJSON o SHP de seccions censals de Barcelona
path_geo = "data/BarcelonaCiutat_SeccionsCensals.csv"
gdf = gpd.read_file(path_geo)

# CSV amb la variable del dia (ex: vulnerabilitat hídrica)
# Ha de tenir: codi_seccio | vulnerabilitat
path_data = "data/vulnerabilitat_dia.csv"
df_val = pd.read_csv(path_data)

In [ ]:
# Si els codis tenen altres noms, ajusta aquesta línia
gdf = gdf.merge(df_val, left_on="CUSEC", right_on="codi_seccio", how="left")

In [ ]:
# Centroid de Barcelona
center = [41.3874, 2.1686]

m = folium.Map(location=center, zoom_start=12, tiles="CartoDB positron")

In [ ]:
# Color scale
colormap = linear.OrRd_09.scale(
    gdf["vulnerabilitat"].min(),
    gdf["vulnerabilitat"].max()
)

colormap.caption = "Vulnerabilitat hídrica"
colormap.add_to(m)

In [ ]:
# Add seccions censals to map
def style_function(feature):
    value = feature["properties"]["vulnerabilitat"]
    if pd.isna(value):
        return {"fillColor": "#cccccc", "color": "black", "weight": 0.3, "fillOpacity": 0.4}
    return {
        "fillColor": colormap(value),
        "color": "black",
        "weight": 0.2,
        "fillOpacity": 0.7
    }

folium.GeoJson(
    gdf,
    name="Seccions censals",
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=["CUSEC", "vulnerabilitat"])
).add_to(m)

m